In [ ]:
# Import standard libraries
import shutil
import os
import time
import subprocess

# Import third-party libraries (Colab-specific)
try:
    from google.colab import drive #type:ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Define utility logging function
def log(message):
    print(f"\n> {message}")

# Install required third-party libraries (Colab only)
log("Installing required libraries for Colab")
subprocess.run(["pip", "install", "XlsxWriter"], check=True)
log("Required libraries have been installed")

# Configuration
FORCE_MOUNT = False
REPOSITORY_PATH = '/content/drive/MyDrive/repositories/ml-playverse'
PROJECT_NAMES = ['boston-housing', 'mnist']
PROCESS_TIMEOUT = 60

# Mount Google Drive
if IN_COLAB:
    log("Mounting Google Drive on Colab\n")
    if FORCE_MOUNT:
        drive.mount("/content/drive", force_remount=True)
    else:
        drive.mount("/content/drive")

    # Wait until mounted
    for _ in range(PROCESS_TIMEOUT):
        if os.path.exists("/content/drive/MyDrive"):
            break
        time.sleep(1)
    else:
        raise TimeoutError("Google Drive mounting timed out")

    log("Google Drive is successfully mounted")
else:
    raise EnvironmentError("This script must run in Google Colab")

# Delete old repo (if it exists)
log("Deleting old repository from Colab")
if os.path.exists(REPOSITORY_PATH):
    shutil.rmtree(REPOSITORY_PATH)
    log("Old repository has been deleted")
else:
    log("Old repository did not exist")

# Ensure parent directory exists
parent_dir = os.path.dirname(REPOSITORY_PATH)
os.makedirs(parent_dir, exist_ok=True)

# Clone GitHub repository into Drive
log("Cloning up-to-date repository from GitHub")
subprocess.run([
    "git", "clone",
    "https://github.com/saeedkhademidoroh/ml-playverse.git",
    REPOSITORY_PATH
], check=True)

# Wait until clone completes
for _ in range(PROCESS_TIMEOUT):
    if os.path.isdir(REPOSITORY_PATH) and os.listdir(REPOSITORY_PATH):
        break
    time.sleep(1)
else:
    raise TimeoutError("Repository cloning timed out")

log("Successfully cloned repository")

# Run main.py in each project
for project in PROJECT_NAMES:
    project_path = os.path.join(REPOSITORY_PATH, project)
    log(f"Processing project: {project}")

    if os.path.exists(project_path):
        os.chdir(project_path)
        log(f"Changed working directory to: {os.getcwd()}")
        log("Running main.py")
        subprocess.run(["python", "main.py"], check=True)
    else:
        log(f"Skipped: {project_path} does not exist")
